In [4]:
import pandas as pd
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, TypedDict
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import re
import pdfplumber

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議
    impact_score: int  # 新增的影響程度分數

# 定義個性化新聞代理類
class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model  # 初始化生成式語言模型
        self.checkpointer = checkpointer  # 初始化檢查點保存器，用於保存狀態
        builder = StateGraph(CustomerState)  # 創建StateGraph並傳入狀態字典類型
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("impact_assessment", self.impact_assessment_node)  # 新增影響評估節點
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "impact_assessment")  # 添加新的邊
        builder.add_edge("impact_assessment", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
            "impact_score": 50,  # 初始化影響程度分數為 50（中等）
        }

    def extract_customer_features_node(self, state: CustomerState):
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="以下所有產品皆為基金類型的產品，請直接列點總結每個客戶特徵，100字內，列點包括投資配置、產品偏好、風險偏好、投資策略、交易狀況、持倉狀況等，並確保生成內容完全來自以下客戶特徵信息，不參考其他來源。請用繁體中文回答。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        state["extracted_features"] = response.content.strip()
        return {"extracted_features": state["extracted_features"]}

    def personalized_news_summary_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="請根據以下新聞內容及客戶特徵生成新聞重點摘要，個人化條列式生成3至5點，與客戶投資產品相關的新聞放在最前面，每點字數不超過30字，總字數少於200字，不要包括任何客戶特徵信息，只能從以下新聞內容生成摘要。請用繁體中文回答"),
            HumanMessage(content=f"客戶特徵: {state['extracted_features']}"),
            HumanMessage(content=state["news_content"])
        ]
        response = self.model.invoke(messages)
        state["news_summary"] = response.content.strip()
        return {"news_summary": state["news_summary"]}

    def investment_advice_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請嚴格基於每位客戶特徵和新聞摘要，列點式生成個人化投資建議，直接給具體建議即可。不要加入其他外部資訊，不要推薦任何產品，建議字數應在200字內。請確保新聞摘要中的資訊有確實反映在投資建議中，投資建議應該考慮到客戶的風險承受能力、投資目標、資產配置偏好，以及當前市場狀況，務必確保每個建議都是實際可行的。請用繁體中文回答。"),
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
            AIMessage(content=f"新聞摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        state["investment_advice_result"] = response.content.strip()
        return {"investment_advice_result": state["investment_advice_result"]}

    def impact_assessment_node(self, state: CustomerState):
        messages = [
        SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請評估新聞對每個客戶的影響程度。給出0到100分的評分，分數越高代表影響越大。"),
        AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
        AIMessage(content=f"新聞摘要: {state['news_summary']}"),
    ]
        response = self.model.invoke(messages)
        impact_text = response.content.strip()

    # 初始化默認值
        score = 20
        explanation = "影響甚小，先給予20分"

    # 檢查是否模型返回了完整的提示而不是结果
        if "0到100分的評分" not in impact_text:
        # 使用正則表達式提取数字評分
            score_match = re.search(r'\b(\d{1,3})\b', impact_text)
            if score_match:
                score = int(score_match.group(1))
            # 保证分數在0到100之间
                score = min(max(score, 0), 100)
                explanation = impact_text[impact_text.find(score_match.group(1)) + len(score_match.group(1)):].strip()
                explanation = explanation.replace('*', '').strip()  # 移除可能的米字號或其他不必要符号
            else:

            # 如果没有提取到有效分数，使用範例值並解釋
                score = 20
                explanation = "影響甚小，先給予20分"

    # 构建影响评分结果，确保没有多余的分数后缀
        state["impact_score"] = f"{score} 分" if explanation else f"{score} 分 - {explanation}"
        return {"impact_score": state["impact_score"]}

    def output_results_node(self, state: CustomerState):
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
            "impact_score": state["impact_score"],  # 包含影響程度分數
        }

# 定義格式化輸出結果的函數
def format_output_for_customer(customer_name, extracted_features, news_summary, investment_advice_result, impact_score):
    return (f"客戶名稱: {customer_name}\n"
            f"提取的客戶特徵:\n{extracted_features}\n\n"
            f"新聞摘要:\n{news_summary}\n\n"
            f"投資建議:\n{investment_advice_result}\n\n"
            f"新聞對客戶的影響程度: {impact_score} 分\n")

# 定義PDF提取摘要的函數
def extract_pdf_summary(pdf_file_path):
    summary = []
    with pdfplumber.open(pdf_file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            summary.append(text[:200] + "...")  # 提取前200個字作為摘要
    return "\n".join(summary)

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
memory = SqliteSaver.from_conn_string(":memory:")

bot = PersonalizedNewsAgent(model=model, checkpointer=memory)
    
thread = {"configurable": {"thread_id": "1"}}

# 讀取Excel表格
df = pd.read_excel("teste.xlsx")

# 從PDF提取新聞內容
pdf_summary = extract_pdf_summary("test.pdf")

# 定義輸入的新聞內容
state = {
    "news_content": pdf_summary
}

# 生成客戶特徵
all_customers_output = []
all_customers_data = []

# 你可以選擇指定要使用的欄位名稱
selected_columns = ['客戶風險等級','近一年總成交量', '股票類型交易量', '債券類型交易量', '平衡類型交易量', '股票類型交易量佔比', '債券類型交易量佔比', '平衡類型交易量佔比'
                    ,'近一年總成交次數','股票類型交易次數','債券類型交易次數','平衡類型交易次數','股票類型交易次數佔比','債券類型交易次數佔比'
                    ,'平衡類型交易次數佔比','庫存量','股票類型庫存量','債券類型庫存量','平衡類型庫存量','股票類型庫存量佔比','債券類型庫存量佔比'
                    ,'平衡類型庫存量佔比','RR1佔比','RR2佔比','RR3佔比','RR4佔比','RR5佔比','近一年REITs的總成交量','近一年大中華股票的總成交量'
                    ,'近一年中國股票的總成交量','近一年巴西股票的總成交量','近一年日本中小型股票的總成交量','近一年日本股票的總成交量'
                    ,'近一年台灣股票的總成交量','近一年全球平衡的總成交量','近一年全球股票的總成交量','近一年全球債的總成交量','近一年印度股票的總成交量'
                    ,'近一年亞太股票的總成交量','近一年亞太區(除日本)股票的總成交量','近一年亞太區股票的總成交量','近一年亞太區貨幣債的總成交量'
                    ,'近一年拉美股票的總成交量','近一年東南亞股票的總成交量','近一年金融股票 的總成交量','近一年南韓股票的總成交量'
                    ,'近一年科技股票的總成交量','近一年美元高息/高收益債券的總成交量','近一年美國股票的總成交量','近一年美債的總成交量'
                    ,'近一年英國股票的總成交量','近一年香港股票的總成交量','RR5佔比','近一年高收益債的總成交量','近一年貨幣型的總成交量'
                    ,'近一年黃金的總成交量','近一年新興市場股票的總成交量','近一年新興市場債的總成交量','近一年新興歐洲股票的總成交量','近一年德國股票的總成交量'
                    ,'近一年歐洲中小型股票的總成交量','近一年歐洲股票的總成交量','近一年歐洲新興市場股票的總成交量','近一年環球股票的總成交量佔比'
                    ,'近一年環球高息/高收益債券的總成交量佔比','近一年環球債券的總成交量佔比','近一年環球新市債的總成交量佔比'
                    ,'近一年環球新興市場股票的總成交量佔比','近一年環球新興市場強勢貨幣債券的總成交量佔比','近一年醫療生技的總成交量佔比'
                    ,'近一年醫療保健股票的總成交量佔比','近一年礦業股票的總成交量佔比','近一年REITs的總成交量佔比','近一年大中華股票的總成交量佔比'
                    ,'近一年中國股票的總成交量佔比','近一年巴西股票的總成交量佔比','近一年日本中小型股票的總成交量佔比'
                    ,'近一年日本股票的總成交量佔比','近一年台灣股票的總成交量佔比','近一年全球平衡的總成交量佔比','近一年全球股票的總成交量佔比'
                    ,'近一年全球債的總成交量佔比','近一年印度股票的總成交量佔比','近一年亞太股票的總成交量佔比','近一年亞太區(除日本)股票的總成交量佔比'
                    ,'近一年亞太區股票的總成交量佔比','近一年亞太區貨幣債的總成交量佔比','近一年拉美股票的總成交量佔比','近一年東南亞股票的總成交量佔比'
                    ,'近一年金融股票 的總成交量佔比','近一年南韓股票的總成交量佔比','近一年科技股票的總成交量佔比','近一年美元高息/高收益債券的總成交量佔比'
                    ,'近一年美國股票的總成交量佔比','近一年美債的總成交量佔比','近一年英國股票的總成交量佔比','近一年香港股票的總成交量佔比'
                    ,'近一年能源股票 的總成交量佔比','近一年高收益債的總成交量佔比','近一年貨幣型的總成交量佔比','近一年黃金的總成交量佔比'
                    ,'近一年新興市場股票的總成交量佔比','近一年新興市場債的總成交量佔比','近一年新興歐洲股票的總成交量佔比','近一年德國股票的總成交量佔比'
                    ,'近一年歐洲中小型股票的總成交量佔比','近一年歐洲股票的總成交量佔比','近一年歐洲新興市場股票的總成交量佔比','近一年環球股票的總成交量佔比'
                    ,'近一年環球高息/高收益債券的總成交量佔比','近一年環球債券的總成交量佔比','近一年環球新市債的總成交量佔比'
                    ,'近一年環球新興市場股票的總成交量佔比','近一年環球新興市場強勢貨幣債券的總成交量佔比','近一年醫療生技的總成交量佔比'
                    ,'近一年醫療保健股票的總成交量佔比','近一年礦業股票的總成交量佔比','REITs的庫存量','大中華股票的庫存量','中國股票的庫存量','巴西股票的庫存量'
                    ,'日本中小型股票的庫存量','日本股票的庫存量','台灣股票的庫存量','全球平衡的庫存量','全球股票的庫存量','全球債的庫存量'
                    ,'印度股票的庫存量','亞太股票的庫存量','亞太區(除日本)股票的庫存量','亞太區股票的庫存量','亞太區貨幣債的庫存量','拉美股票的庫存量'
                    ,'東南亞股票的庫存量','金融股票 的庫存量','南韓股票的庫存量','科技股票的庫存量','美元高息/高收益債券的庫存量','美國股票的庫存量'
                    ,'美債的庫存量','英國股票的庫存量','香港股票的庫存量','能源股票 的庫存量','高收益債的庫存量','貨幣型的庫存量','黃金的庫存量'
                    ,'新興市場股票的庫存量','新興市場債的庫存量','新興歐洲股票的庫存量','德國股票的庫存量','歐洲中小型股票的庫存量','歐洲股票的庫存量'
                    ,'歐洲新興市場股票的庫存量','環球股票的庫存量','環球高息/高收益債券的庫存量','環球債券的庫存量','環球新市債的庫存量'
                    ,'環球新興市場股票的庫存量','環球新興市場強勢貨幣債券的庫存量','醫療生技的庫存量','醫療保健股票的庫存量','礦業股票的庫存量'
                   ]

# 迭代每個客戶，生成特徵摘要和投資建議
for index, row in df.iterrows():
    customer_name = row['ID']
    
    # 根據選擇的欄位來生成客戶特徵
    customer_features = []
    for col in selected_columns:
        if pd.notna(row[col]):  # 檢查該欄位是否有數值
            feature = f"{col}: {row[col]}"
            customer_features.append(feature)
    
    # 更新狀態
    state["customer_features"] = customer_features

    # 執行圖形流程
    result = bot.graph.invoke(state, thread)

    # 檢查生成的投資建議是否為空，如果是空的，給予警告並記錄
    if not result["investment_advice_result"]:
        print(f"警告: {customer_name} 的投資建議生成為空！")

    # 格式化輸出結果
    customer_output = format_output_for_customer(
        customer_name, 
        result["extracted_features"], 
        result["news_summary"],
        result["investment_advice_result"], 
        result["impact_score"]  # 添加影響程度分數
    )
    
    all_customers_output.append(customer_output)

    all_customers_data.append({
        "客戶": customer_name,
        "客戶特徵": result["extracted_features"],
        "新聞摘要": result["news_summary"],
        "投資建議": result["investment_advice_result"],
        "影響程度分數": result["impact_score"]  # 新增的影響程度分數列
    })

# 創建DataFrame並輸出到Excel
output_df = pd.DataFrame(all_customers_data)
with pd.ExcelWriter("Personalized_Investment_Advice_Output.xlsx", engine="openpyxl") as writer:
    output_df.to_excel(writer, sheet_name="Sheet1", index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # 調整欄位寬度
    for i, col in enumerate(output_df.columns):
        max_width = max(output_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print("Excel 文件已生成並保存，請檢查結果。")


Excel 文件已生成並保存，請檢查結果。
